## Options for the generated tetrahedra-grid

In this notebook we look over the options for the generation of the interpolation grid. The main executable *test_gorilla_main.x* takes as input for the grid generation the fortran namelist provided in the *tetra_grid.inp* file. One can per hand change the default values of those namelist-variables directly and start the exectuable afterwards by hand.

However, via the the *f90nml* package one can control the grid-options in a python script which then allows to integrate *GORILLA* in existing python routines. This is what we will demonstrate here.

### Using *f90nml* to alter the namelist

First we load the *f90nml* package after installing it e.g. using pip. Additionally we load the default module *os* (this is just so that we can automatically specify our working directory).

In [288]:
import f90nml
import os

path_jupyter = os.getcwd()

The namelist we are looking for (*tetra_grid_nml*) is located in the file *tetra_grid.inp*, which can be found in the *INPUT* folder. So we provide the correct path to that file and read it into a python dictionary. Make sure to set the *end_comma* property of the generated namelist-object to *True*.

In [289]:
filename= path_jupyter + '/../INPUT/tetra_grid.inp'
tetra_grid = f90nml.read(filename)
tetra_grid.end_comma = True
print(tetra_grid)

&tetra_grid_nml
    n1 = 100,
    n2 = 40,
    n3 = 40,
    grid_kind = 3,
    g_file_filename = 'MHD_EQUILIBRIA/g_file_for_test',
    convex_wall_filename = 'MHD_EQUILIBRIA/convex_wall_for_test.dat',
    netcdf_filename = 'MHD_EQUILIBRIA/netcdf_file_for_test.nc',
    boole_n_field_periods = .true.,
    n_field_periods_manual = 1,
    sfc_s_min = 0.1,
    theta_geom_flux = 1,
    theta0_at_xpoint = .true.,
    boole_write_mesh_obj = .false.,
    filename_mesh_rphiz = 'mesh_rphiz.obj',
    filename_mesh_sthetaphi = 'mesh_sthetaphi.obj',
/


You can now access and change the values of the listed *key:value pairs* in that dictonary by first indexing the wanted namelist and then the keyname:

In [290]:
print('The current number of grid points n1 is ' + str(tetra_grid['tetra_grid_nml']['n1']))

tetra_grid['tetra_grid_nml']['n1'] = 101

print('The new value of n1 is ' + str(tetra_grid['tetra_grid_nml']['n1']))

The current number of grid points n1 is 100
The new value of n1 is 101


After we have altered the desired options, we write our changed namelist to a new namelist-file (this is done to preserve the comments in the original file as well as the default settings). The final file should be located in the same folder as the main excutable *test_gorilla_main.x* so that it can then properly acess the file.

In [291]:
new_filename = path_jupyter + '/../tetra_grid.inp'
tetra_grid.write(new_filename, force = True);

The commments in the original *.inp* file explain the different options of the namelist *tetra_grid_nml*. We will cover them now step by step.

### Basic grid options

The first three entries specify the **number of grid points** for the different coordinate directions.

In [292]:
#Rectangular: nR, Field-aligned: ns
tetra_grid['tetra_grid_nml']['n1'] = 100

#Rectangular: nphi, Field-aligned: nphi 
tetra_grid['tetra_grid_nml']['n2'] = 40

#Rectangular: nZ, Field-aligned: ntheta 
tetra_grid['tetra_grid_nml']['n3'] = 40

Following that, the actual **grid kind** used can be changed.

In [293]:
#1 ... rectangular grid for axisymmetric EFIT data (g-file)
#2 ... field-aligned grid for axisymmetric EFIT data (g-file)
#3 ... field-aligned grid for non-axisymmetric VMEC (netcdf)
tetra_grid['tetra_grid_nml']['grid_kind'] = 3

### Source files of fields

*GORILLA* needs the electromagnetic fields as input to perform the trajectory integration. Additionally, the fields are also needed to generate the field-aligned grid. Therefore the corresponding **MHD equilibrium filenames** are provided via the namelist.

In [294]:
tetra_grid['tetra_grid_nml']['g_file_filename'] = 'MHD_EQUILIBRIA/g_file_for_test' 
tetra_grid['tetra_grid_nml']['convex_wall_filename'] = 'MHD_EQUILIBRIA/convex_wall_for_test.dat' 
tetra_grid['tetra_grid_nml']['netcdf_filename'] = 'MHD_EQUILIBRIA/netcdf_file_for_test.nc' 

### Advanced grid options

By default the **periods of the field** is determined automatically by *GORILLA*. However, it may be also set manually.

In [295]:
#True ... number of field periods is selected automatically (Tokamak = 1, Stellarator depending on VMEC equilibrium)
#False ... number of field periods is selected manually (see below)
tetra_grid['tetra_grid_nml']['boole_n_field_periods'] = True 

#Number of field periods (manual)
tetra_grid['tetra_grid_nml']['n_field_periods_manual'] = 1

During the generation of the field aligned grid with Symmetry Flux Coodinates, a small region around the magnetic axis is left out, due to insufficient precission of conversion functions for this region. The grid is therefore only generated down to a **minimal value of the flux fluxcoordinate s**.

In [296]:
tetra_grid['tetra_grid_nml']['sfc_s_min'] = 0.1

By default the aligned grid is generated by having equidistant points in SFC (s,$\vartheta$,$\phi$). However, for the flux poloidal angle $\vartheta$ there is the option to have the points **equidistant in terms of the geometrical poloidal angle $\theta$** instead. This avoids undesired deformations of the grid in real space.

In [297]:
#1 ... theta scaling in symmetry flux coordinates
#2 ... theta scaling in geometrical theta angle
tetra_grid['tetra_grid_nml']['theta_geom_flux'] = 1

No matter which option is chosen, an **origin for the poloidal angle variable** has to be set. Either the angle is measured from the line between magnetic axis and X-point of the magnetic field or just from the right horizontal going out from the magnetic axis.

In [298]:
#True $\theta$-variable starts at the line between O- and X-Point
#False $\theta$-variable starts at the line between O-Point and intersection between O-Point-[1,0]-straight-line and separatrix
tetra_grid['tetra_grid_nml']['theta0_at_xpoint'] = True

### Save generated grid as object files

*GORILLA* generates the grid (vertex positions plus assignment to the tetrahedra) everytime the executable is called. It therefore only exists temporary during runtime. The last settings of *tetra_grid_nml* allow to **save the generated grid** into specified object files.

In [299]:
#Switch for writing object file with mesh data
tetra_grid['tetra_grid_nml']['boole_write_mesh_obj'] = False

#Filename for mesh object file in cylindrical coordintes
tetra_grid['tetra_grid_nml']['filename_mesh_rphiz'] = 'mesh_rphiz.obj' 

#Filename for mesh object file in symmetry flux coordintes
tetra_grid['tetra_grid_nml']['filename_mesh_sthetaphi'] = 'mesh_sthetaphi.obj' 

Finally, we again save all our changes into the *.inp* file which will be used by *GORILLA*.

In [300]:
tetra_grid.write(new_filename, force= True);